In [1]:
import sys  
sys.path.insert(0, 'diaa/')

In [2]:
from collections import Counter
from main import main
import pandas as pd
import json

In [37]:
main_file = "FewShot(5)-NoICL"
folder = "notebook-docred-ner"

In [38]:
OUTPUT_CHATGPT = f"../{folder}/ner_annotation_by_chatgpt/{main_file}.json"
chatgpt_annotation = pd.read_json(OUTPUT_CHATGPT)
chatgpt_annotation = chatgpt_annotation.sort_values(by=["idx"])
chatgpt_annotation = chatgpt_annotation.reset_index(drop=True)

In [39]:
MAIN_DATASET = "../dataset/docred-dataset.json".format(main_file)
main_dataset = pd.read_json(MAIN_DATASET)
main_dataset = main_dataset.sort_values(by=["id"])

In [40]:
labels = {
    "PERSON":1,
    "LOCATION":2,
    "ORGANIZATION":3,
    "NORP":4,
    "FACILITY":5,
    "PRODUCT":6,
    "EVENT":7,
    "WORK_OF_ART":8,
    "LANGUAGE":9,
    "LAW":10,
    "ROLE":11,
    "TITLE":12
}

In [42]:
# For DocRED ChatGPT Experiment
all_annotation = []
for idx, row in chatgpt_annotation.iterrows():
    gold = [{"label": labels[item["label"]], "start_offset": item["position"]["start_offset"], "end_offset": item["position"]["end_offset"], "user": "gold"} for item in main_dataset.iloc[idx]["entities"] if item["label"] in ['PERSON','LOCATION','ORGANIZATION']]
    chatgpt_our_selection = [item for item in row["chatgpt_ent"] if item["label"] in ['PERSON','LOCATION','ORGANIZATION']]
    chatgpt = [{"label": labels[item["label"]], "start_offset": item["position"]["start_offset"], "end_offset": item["position"]["end_offset"], "user": "chatgpt"} for item in chatgpt_our_selection]
    all_annotation.append({
        "id": idx,
        "text": row["text"],
        "annotations": combined
    })

In [43]:
main([all_annotation])

# Inter-Annotator Agreement Report

* Token-based F1 agreement

## Project Setup

* 2 annotators: gold, chatgpt
* 200 agreement documents
* 3 labels

## Agreement per Document

| Document   |   Mean F1 |   SD F1 |
|------------|-----------|---------|
| 55.ann     |     0.538 |   0.000 |
| 57.ann     |     0.398 |   0.000 |
| 73.ann     |     0.606 |   0.000 |
| 16.ann     |     0.419 |   0.000 |
| 61.ann     |     0.685 |   0.000 |
| 97.ann     |     0.536 |   0.000 |
| 63.ann     |     0.655 |   0.000 |
| 98.ann     |     0.483 |   0.000 |
| 58.ann     |     0.636 |   0.000 |
| 130.ann    |     0.515 |   0.000 |
| 92.ann     |     0.288 |   0.000 |
| 118.ann    |     0.583 |   0.000 |
| 48.ann     |     0.700 |   0.000 |
| 79.ann     |     0.462 |   0.000 |
| 135.ann    |     0.487 |   0.000 |
| 11.ann     |     0.478 |   0.000 |
| 78.ann     |     0.684 |   0.000 |
| 144.ann    |     0.575 |   0.000 |
| 42.ann     |     0.741 |   0.000 |
| 40.ann     |     0.549 |   0.000 |
| 166.ann

{'kappa': 0.14312361069835003,
 'fleiss': 0.14312361069835003,
 'alpha': 0.05091324748525916,
 'scotts': 0.05081771786749209}

In [22]:
percent_correct_overall = []
by_label_f1 = {
    "PERSON":[],
    "LOCATION":[],
    "ORGANIZATION":[],
    "NORP":[],
    "FACILITY":[],
    "PRODUCT":[],
    "EVENT":[],
    "WORK_OF_ART":[],
    "LANGUAGE":[],
    "LAW":[],
    "ROLE":[],
    "TITLE":[]
}
by_label_percent = {
    "PERSON":[],
    "LOCATION":[],
    "ORGANIZATION":[],
    "NORP":[],
    "FACILITY":[],
    "PRODUCT":[],
    "EVENT":[],
    "WORK_OF_ART":[],
    "LANGUAGE":[],
    "LAW":[],
    "ROLE":[],
    "TITLE":[]
}

entity_count = {
    "PERSON":[],
    "LOCATION":[],
    "ORGANIZATION":[],
    "NORP":[],
    "FACILITY":[],
    "PRODUCT":[],
    "EVENT":[],
    "WORK_OF_ART":[],
    "LANGUAGE":[],
    "LAW":[],
    "ROLE":[],
    "TITLE":[]
}

for idx, row in chatgpt_annotation.iterrows():
    gold_full = [(item["label"],item["position"]["start_offset"],item["position"]["end_offset"]) for item in list(row["gold_ent"]) if item["mention"].upper()!="COVID-19"]
    cgpt_full = [(item["label"],item["position"]["start_offset"],item["position"]["end_offset"]) for item in list(row["chatgpt_ent"]) if item["mention"].upper()!="COVID-19"]
    percent_correct_overall.append(round((len([item for item in cgpt_full if item in gold_full])/len(cgpt_full)),2))
    for key in by_label_f1:
        gold = [item for item in gold_full if item[0]==key]
        cgpt = [item for item in cgpt_full if item[0]==key]
        entity_count[key].append(len(gold))
        agreed = list((Counter(gold) & Counter(cgpt)).elements())
        disagreed = list((Counter(gold) | Counter(cgpt)).elements())
        agreed_len = len(agreed)
        disagreed_len = len(disagreed)

        try:
            precision = float(agreed_len / (agreed_len+disagreed_len))
            recall = float(agreed_len / (agreed_len+disagreed_len))
        except:
            precision = 0
            recall = 0
        try:
            f1 = (2 * precision * recall) / float(precision + recall)
        except:
            f1 = 0
        try:
            percent_correct = round((len([item for item in cgpt if item in gold])/len(cgpt)),2)
        except:
            percent_correct = 0
        by_label_f1[key].append(f1)
        by_label_percent[key].append(percent_correct)

In [132]:
for key in entity_count:
    print("Entity Count for {} is: {}".format(key,sum(entity_count[key])))

Entity Count for PERSON is: 1646
Entity Count for LOCATION is: 1157
Entity Count for ORGANIZATION is: 1624
Entity Count for NORP is: 114
Entity Count for FACILITY is: 208
Entity Count for PRODUCT is: 72
Entity Count for EVENT is: 386
Entity Count for WORK_OF_ART is: 7
Entity Count for LANGUAGE is: 0
Entity Count for LAW is: 62
Entity Count for ROLE is: 485
Entity Count for TITLE is: 300


In [133]:
for key in by_label_f1:
    print("Average F1 Score for Label {} is: {}".format(key,round((sum(by_label_f1[key])/len(by_label_f1[key])),2)))

Average F1 Score for Label PERSON is: 0.05
Average F1 Score for Label LOCATION is: 0.15
Average F1 Score for Label ORGANIZATION is: 0.03
Average F1 Score for Label NORP is: 0.01
Average F1 Score for Label FACILITY is: 0.0
Average F1 Score for Label PRODUCT is: 0.0
Average F1 Score for Label EVENT is: 0.02
Average F1 Score for Label WORK_OF_ART is: 0.0
Average F1 Score for Label LANGUAGE is: 0.0
Average F1 Score for Label LAW is: 0.01
Average F1 Score for Label ROLE is: 0.02
Average F1 Score for Label TITLE is: 0.0


In [134]:
percent = []
for key in by_label_percent:
    print("Average % of Correctness for Label {} is: {}".format(key,round((sum(by_label_percent[key])/len(by_label_percent[key])),2)))
    percent.append(round((sum(by_label_percent[key])/len(by_label_percent[key])),2))
print(percent)
print(sum(percent)/12)

Average % of Correctness for Label PERSON is: 0.2
Average % of Correctness for Label LOCATION is: 0.34
Average % of Correctness for Label ORGANIZATION is: 0.07
Average % of Correctness for Label NORP is: 0.01
Average % of Correctness for Label FACILITY is: 0.01
Average % of Correctness for Label PRODUCT is: 0.01
Average % of Correctness for Label EVENT is: 0.07
Average % of Correctness for Label WORK_OF_ART is: 0.0
Average % of Correctness for Label LANGUAGE is: 0.0
Average % of Correctness for Label LAW is: 0.03
Average % of Correctness for Label ROLE is: 0.04
Average % of Correctness for Label TITLE is: 0.01
[0.2, 0.34, 0.07, 0.01, 0.01, 0.01, 0.07, 0.0, 0.0, 0.03, 0.04, 0.01]
0.06583333333333335


In [135]:
print(sum(percent_correct_overall)/(len(percent_correct_overall)))

0.19054999999999991
